In [1]:
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError
from azure.search.documents import SearchClient
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from random import randint
from typing import Dict, Tuple
from unidecode import unidecode
import config
import glob
import os.path
import re
search_endpoint = "https://ai-search-acpr-hackathon.search.windows.net" # Add your Azure AI Search endpoint here
search_key = config.azure_search_key  # Add your Azure AI Search admin key here
search_index_name = "equipe-6-faq-v1" # Add your Azure AI Search index name here

os.environ["SearchEndpoint"] = search_endpoint
os.environ["SearchKey"] = search_key
os.environ['SearchIndex'] = search_index_name

search_endpoint = "https://ai-search-acpr-hackathon.search.windows.net"
search_credential = AzureKeyCredential("SEARCH_API_KEY")
search_client = SearchClient(
  endpoint=search_endpoint,
  index_name=search_index_name,
  credential=search_credential,
)


In [2]:

def compress_and_clean(text: str) -> str:
    """
    Compress and clean a text.

    Use the Snowball stemmer to stem words and remove all special characters, as the LLM does not give a damn.
    """
    text = text.replace("\\", "")  # Remove all backslashes
    text = re.sub(r":[a-z]*:", "", text)  # Remove all :unselected: and :selected: tags
    text = re.sub(r"<!--[^<>]*-->", "", text)  # Remove all comments
    return text


def data(content: str, source_uri: str, title: str, iterator: int) -> dict[str, str]:
    """
    Generate a data object for the search index.

    Use deterministic ID to avoid duplicates after a new run. Remove all special characters from title.
    """
    return {
        "content": content,
        "id": f"{'_'.join(re.sub('[^a-z0-9]', ' ', unidecode(source_uri).lower()).split())}-{iterator}",
        "source_uri": unidecode(source_uri).lower(),
        "title": ' '.join(re.sub('[^a-z0-9]', ' ', unidecode(title).lower()).split()),
    }

In [3]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.vectorstores.azuresearch import AzureSearch
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import Chroma
import glob

In [4]:
list_file_path = glob.glob("./base_de_connaissance/*")
list_file_path

['./base_de_connaissance\\1_JD.txt',
 './base_de_connaissance\\2_JD.txt',
 './base_de_connaissance\\3_JD.txt',
 './base_de_connaissance\\4_JD.txt',
 './base_de_connaissance\\5_JD.txt',
 './base_de_connaissance\\6_JD.txt',
 './base_de_connaissance\\7_JD.txt',
 './base_de_connaissance\\base_de_connaissance.zip',
 './base_de_connaissance\\Question 1.txt',
 './base_de_connaissance\\Question 2.txt',
 './base_de_connaissance\\Question 3.txt',
 './base_de_connaissance\\Question 4.txt',
 './base_de_connaissance\\Question 5.txt',
 './base_de_connaissance\\Question 6.txt',
 './base_de_connaissance\\Question 7.txt',
 './base_de_connaissance\\Question 8.txt',
 './base_de_connaissance\\Question 9.txt']

In [5]:

# file_path = list_file_path[0]
# loader = TextLoader(file_path)
# documents = loader.load()


loader = DirectoryLoader('./base_de_connaissance/', glob="*.txt", loader_cls=TextLoader, loader_kwargs={'autodetect_encoding': True})

documents = loader.load()
# text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=0)
# docs = text_splitter.split_documents(documents)

# Add documents to Azure Search
# acs.add_documents(documents=docs)




In [6]:
os.environ["OPENAI_API_TYPE"] = "azure"
# os.environ["OPENAI_API_BASE"] = "https://francecentral-openai.openai.azure.com"
os.environ["OPENAI_API_KEY"] = config.azure_ada_key
os.environ["OPENAI_API_VERSION"] = "2023-05-15"   


os.environ["AZURE_OPENAI_ENDPOINT"] = "https://francecentral-openai.openai.azure.com"
os.environ["AZURE_OPENAI_API_KEY"] = config.azure_ada_key


In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=256,
    chunk_overlap=128,
    length_function=len,
    add_start_index=True)

chunks = text_splitter.split_documents(documents)

embeddings = AzureOpenAIEmbeddings(deployment="ada-002")        


In [8]:
chunks


[Document(page_content='Question : Quel est le nombre minimal d’adhérent d’une SAM ?', metadata={'source': 'base_de_connaissance\\1_JD.txt', 'start_index': 0}),
 Document(page_content="Réponse : D’après l’article R322-47§2 du code des assurances, le nombre minimal d’adhérent est de 500. Il est réduit à 7 pour les organismes mentionnés à l'article L. 771-1 du code rural et de la pêche maritime.", metadata={'source': 'base_de_connaissance\\1_JD.txt', 'start_index': 62}),
 Document(page_content='Question : Quelles sont les modalités de convocation d’une assemblée générale d’une SAM ?', metadata={'source': 'base_de_connaissance\\2_JD.txt', 'start_index': 0}),
 Document(page_content='Réponse : Ces modalités sont présentées au sein de l’article R322-59 du code des assurances. Il y est en particulier précisé que :\nLes statuts indiquent les conditions dans lesquelles est faite la convocation aux assemblées générales.', metadata={'source': 'base_de_connaissance\\2_JD.txt', 'start_index': 91}),

In [9]:
# azure_vector_store = AzureSearch(azure_search_endpoint=search_endpoint, 
#                                  azure_search_key=search_key,
#                                  index_name=search_index_name, 
#                                  embedding_function=embeddings.embed_query)

# load it into Chroma
db = Chroma.from_documents(chunks, embedding= embeddings)


c:\Users\phile\local\Suptech-sprint\.conda\Lib\site-packages\onnxruntime\capi\onnxruntime_validation.py:26: UserWarning: Unsupported Windows version (11). ONNX Runtime supports Windows 10 and above, only.
  warnings.warn(


In [10]:
# azure_vector_store.add_texts(chunks)


In [11]:
# azure_vector_store.add_documents(documents = chunks)


In [14]:
query = "Quelle part du risque des sociétés d'assurance mutuelle membres d'une union de sociétés d'assurance mutuelle est réassurée par l'union ?" # OK n°1
query = "Qu'est-ce que l'ajustement symétrique ?" # OK n°1
query = "Quelles sont les exigences sur les hypothèses portant sur les futures décisions de gestion dans le calcul des provisions prudentielles ?" # OK n°1
query = "De quoi sont constitués les fonds propres auxiliaires ?" # OK n°1
# query = "En quoi consiste la méthode 2 du calcul de la marge de risque ?" # OK n°1
# query = "Quelle hypothèse est introduite dans la deuxième méthode de calcul de la marge de risque ?" # OK n°1
# query = "Comment est calculé l'ajustement symétrique ?" # OK n°1
# query = "Une évaluation interne des risques et de la solvabilité est elle obligatoire ?"# KO
# query = "Quelles types d'évaluations doivent être menées dans le cadre d'une évaluation interne des risques et de la solvabilité ?"# OK
# query = "Quelles sont les attendus d'un ORSA?"# OK
# query = "A quelle fréquence l'ORSA doit il être réalisé?" # KO
# query = "A quelle date le rapport ORSA doit il être remis aux autorités de contrôle?"# OK
# query = "Existe-t-il un délai de remise du rapprot ORSA?"# OK
# query = "A quelle date le rapport régulier au contrôleur  doit il être remis aux autorités de contrôle?"# OK
# query = "Existe-t-il un délai de remise du rapport régulier au contrôleur ?"# OK
# query = "Quel est le nombre minimal de membre de l'assemblée générale d'une SAM"# KO
# query = "Un organisme peut-il ne pas remettre des QRT?"# KO
query = "Une remise annuelle du RSR est-elle obligatoire?"# OK n°2
# 14 / 18 OK
docs = db.similarity_search(query)

print(docs[0])
print(docs[1])
print(docs[2])
print(docs[3])


page_content='Question: Les remises trimestrielles des états quantitatifs S2 sont elles obligatoires?' metadata={'source': 'base_de_connaissance\\Question 9.txt', 'start_index': 0}
page_content="Question: A quelle fréquence le RSR doit il êtrce remis à l'autorité de contrôle ?" metadata={'source': 'base_de_connaissance\\Question 6.txt', 'start_index': 0}
page_content="Question: Un ORSA annuel est-il obligatoire?\n\nRéponse: Selon l'article R354-3-4 du code des assurances, les entreprises procèdent à cette évaluation interne au moins une fois par an et en cas d'évolution notable de leur profil de risque." metadata={'source': 'base_de_connaissance\\Question 3.txt', 'start_index': 0}
page_content="Question: Quand le RSR doit il être remis à l'autorité de contrôle ?\n\nRéponse: Selon l'article 312 du RD, le le rapport régulier au contrôleur doit être transmis au plus tard 14 semaines après la clôture de l'exercice financier de l'entreprise." metadata={'source': 'base_de_connaissance\\Quest

In [ ]:
# https://codepen.io/t_afif/pen/ExXyXpB




In [ ]:
body {
  font-family: monospace;
  font-weight: 700;
  font-size:30px;
  padding:20px;
  background:#005F6B;
  color:#00DFFC
}

.type {
  color:#0000;
  background:
    linear-gradient(-90deg,#00DFFC 5px,#0000 0) 10px 0,
    linear-gradient(#00DFFC 0 0) 0 0;
  background-size:calc(var(--n)*1ch) 200%;
  -webkit-background-clip:padding-box,text;
  background-clip:padding-box,text;
  background-repeat:no-repeat;
  animation: 
   b .7s infinite steps(1),   
   t calc(var(--n)*.05s) steps(var(--n)) forwards;
}

@keyframes t{
  from {background-size:0 200%}
}
@keyframes b{
  50% {background-position:0 -100%,0 0}
}

In [ ]:
Contrôleur : <span class="type" style="--n:1000">
  

De quoi sont constitués les fonds propres auxiliaires ?<br/>

Eurêka : Voici la question la plus ressemblante dans la FAQ <br/>

Question 6 :  Comment sont calculés les fonds propres prudentielles ? <br/>
D’après l’articles R351-18 du code des assurances, les fonds propres prudentiels sont constitués de l’excédent des actifs par rapport aux passifs prudentiels auxquels sont retirés les actions propres, les passifs subordonnés et les fonds propres auxiliaires hors bilan.
Les fonds propres auxiliaires sont constitués d’après l’article R351-19 du code des assurances d'éléments, autres que les fonds propres de base, qui peuvent être appelés pour absorber des pertes.





</span>